# Niche Fine-Tuning with Unsloth

This notebook demonstrates how to fine-tune Llama 3 8B on your custom niche dataset.

**Prerequisites:**
1. Upload your `dataset.json` file to the Colab content directory (folder icon on left).
2. Runtime -> Change runtime type -> GPU (T4 is fine).

In [ ]:
%%capture
# Install Unsloth, Xformers, and TRL
import torch
major_version, minor_version = torch.cuda.get_device_capability()
if major_version >= 8:
    !pip install "unsloth[colab-ampere] @ git+https://github.com/unslothai/unsloth.git"
else:
    !pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"

!pip install "git+https://github.com/huggingface/transformers.git"
!pip install "git+https://github.com/huggingface/trl.git"

In [ ]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset

# 1. Configuration
max_seq_length = 2048
dtype = None # Auto detection
load_in_4bit = True

# 2. Load Model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# 3. Add LoRA Adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

In [ ]:
# 4. Load Data
# Ensure dataset.json is uploaded!
try:
    dataset = load_dataset("json", data_files="dataset.json", split="train")
    print(f"Loaded {len(dataset)} examples")
except Exception as e:
    print("Error loading dataset.json. Did you upload it?")
    print(e)

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + tokenizer.eos_token
        texts.append(text)
    return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)

In [ ]:
# 5. Train
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # Adjust based on data size (e.g., 1 epoch)
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer.train()

In [ ]:
# 6. Inference / Test
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[    
    alpaca_prompt.format(
        "How do I use this library?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

In [ ]:
# 7. Save and Download
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

!zip -r lora_model.zip lora_model
from google.colab import files
files.download('lora_model.zip')